# HSE Summer School NLP&DA

![](https://www.hse.ru/data/2014/06/25/1309038576/logo_hse_cmyk_e.jpg)

Учебный проект по теме "Тональность отношений субъектов (именованных сущностей)", предполагающий разработку участниками школы под руководством тьюторов программных средств, решающих задачу определения мнения сторон о различных событиях, освещаемых в новостях

Нужен алгоритм, который по выделенным сущностям может найти пары этих сущностей в тексте и вырезать соответствующие куски между ними + несколько слов справа-слева. Возможно, имеет смысл идти от N-ой выделенной сущности до N+2, и включать всё до неё. 

Чем может помочь синтаксический анализ?

Чем поможет морфологический анализ?

In [1]:
import sys 
sys.path.append("/Users/dmitrys/anaconda2/lib/python2.7/site-packages/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymorphy2
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

import pymorphy2
morph = pymorphy2.MorphAnalyzer()


from pymystem3 import Mystem

m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

import string
%matplotlib inline

In [2]:
def commit():
    ! git add NLP_summerschool.ipynb
    ! git commit -m "changed while loop"
    ! git push -u origin master

In [3]:
commit()

[master 35f3a4d] changed while loop
 1 file changed, 96 insertions(+), 196 deletions(-)
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 842 bytes | 0 bytes/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/DmitrySerg/HSE-NLP-Summer-School.git
   598e08d..35f3a4d  master -> master
Branch master set up to track remote branch master from origin.


## Utility functions

In [3]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d

def transformAnnotation(number):
    """
    Given number loads txt file with annotation 
    Returns DataFrame with transformed annotation
    """
    with open("Texts/art{}.ann".format(number)) as f:
        d = f.read()

    d = d.split("\n")

    for i in range(len(d)):
        d[i] = d[i].split("\t")

    d = pd.DataFrame(d)
    d.drop([0], axis=1, inplace=True)
    d = pd.concat([d, pd.DataFrame(d[1].apply(lambda x: x.split()).tolist())], axis=1)
    d.columns = ["to_delete", "entity", "entity_car", "pos_1", "pos_2"]
    d.drop(["to_delete"], axis=1, inplace=True)
    d["entity"] = d["entity"].apply(lambda x: x.strip("\r"))
    d["entity"] = d["entity"].apply(lambda x: x.decode("utf8"))
    
    d = d[~d.entity.isin(["Unknown", "Author"])].entity.reset_index(drop=True)
    
    return d

def transformAnswer(number):
    """
    Given number loads txt file with answer 
    Returns DataFrame with transformed answer
    """
    answ = loadAnswer(number)
    answ = answ.split("\n")

    for i in range(len(answ)):
        answ[i] = answ[i].split(",")

    answ = pd.DataFrame(answ)
    answ.columns = ["entity_1", "entity_2", "attitude", "time"]
    answ.dropna(inplace=True)
    answ.time = answ.time.apply(lambda x: x.strip("\r"))
    
    return answ

def loadRuSentiLex():
    """
    Loads the RuSentiLex 2017 dictionary
    Returns data frame with ["word", "tag", "word_lemmatized", "tone", "certainty"]
    """
    with open("RuSentiLex2017_revised_2.txt") as f:
        Rusentilex = f.read().decode("cp1251").encode("utf8")
        Rusentilex = Rusentilex[1510:]

    Rusentilex = Rusentilex.split("\n")
    for i, item in enumerate(Rusentilex):
        Rusentilex[i] = item.split(",")

    for i in Rusentilex:
        if len(i) < 5:
            Rusentilex.remove(i)

    Rusentilex = pd.DataFrame(Rusentilex)
    Rusentilex.drop([5, 6, 7], axis=1, inplace=True)
    Rusentilex.columns = ["word", "tag", "word_lemmatized", "tone", "certainty"]
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip('\r'))
    
    
    Rusentilex["tone"] = Rusentilex["tone"].apply(lambda x: x.strip(' '))
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip(' '))
    
    return Rusentilex

def cleanString(myString):
    return myString.translate(None, string.punctuation).decode('utf-8')

def getSentimentCertainity(word):
    word = lemmatize(word)
    try:
        tone, certainty =  Rusentilex[["tone", "certainty"]][Rusentilex.word.isin([word])].values[0]
    except:
        tone, certainty = np.NaN, np.NaN
    return tone, certainty

![](http://cathyreisenwitz.com/wp-content/uploads/2016/01/no.jpg)

Словарь РуСентиЛекс

Структура: 
- 1 слово или словосочетание,
- 2 Часть речи или синтаксический тип группы,
- 3 слово или словосочетание в лемматизированной форме, 
- 4 Тональность: позитивная (positive), негативная(negative), нейтральная (neutral) или неопределеная оценка, зависит от контекста (positive/negative),
- 5 Источник: оценка (opinion), чувство (feeling), факт (fact),
- 6 Если тональность отличается для разных значений многозначного слова, то перечисляются все значения слова по тезаурусу РуТез и дается отсылка на сооветствующее понятие - имя понятия в кавычках.

In [4]:
Rusentilex = loadRuSentiLex()

In [5]:
Rusentilex.head()

,word,tag,word_lemmatized,tone,certainty
0,аборт,Noun,аборт,negative,fact
1,абортивный,Adj,абортивный,negative,fact
2,абракадабра,Noun,абракадабра,negative,opinion
3,абсурд,Noun,абсурд,negative,opinion
4,абсурдность,Noun,абсурдность,negative,opinion


In [6]:
Rusentilex.certainty.value_counts()

opinion             9269
fact                5297
feeling             1536
operator              41
negative               2
positive               2
fact В ТЕЗ empty       1
fact В ТЕЗ EMPTY       1
opinegativenion        1
Name: certainty, dtype: int64

In [7]:
# x = ""
# for i in range(50):
#     try:
#         x+="=============================\n"
#         x+="=============================\n"
#         x+="=============================\n"
#         x+= loadAnswer(i)
#     except:
#         continue
# with open("FULLTEXT.txt", "w") as f:
#     f.write(x)

In [8]:
getSentimentCertainity(u"абортами".encode("utf8"))

('negative', 'fact')

In [9]:
def Text_to_dict(number):
    t = loadText(number)
    text_dict = t.split("\n\n")
    text_dict = {i:parag for i, parag in enumerate(text_dict)}
    sentence_dict = {i:{} for i in range(len(text_dict))}

    for key, paragraph in text_dict.iteritems():
        paragraph = paragraph.split("{Author, Unknown}")
        for sent_numbet, sentence in enumerate(paragraph):

            sentence = cleanString(sentence)
            if len(sentence) != 0:
                sentence_dict[key][sent_numbet] = sentence
                #tknzr.tokenize()
    return sentence_dict

In [10]:
# for key, value in t.iteritems():
#     print("paragraph number", key)
#     for another_key, another_value in value.iteritems():
        
#         print("sentence_number", another_key)
#         for word in another_value:
#             print word

In [11]:
text = Text_to_dict(10)
entities = transformAnnotation(10)
answer = transformAnswer(10) 

In [12]:
def getEntityPositions(text, entities):
    """
    This bad boy finds (hopefully) all entities in the text
    Collects their exact locations and returns a neat Dataframe with them
    """
    
    ENTITIES = pd.DataFrame(columns=["entity", "paragraph", "sentence", "loc_start", "loc_end"])
    
    for paragraph in text:
        for sentence in text[paragraph]:
            for entity in list(entities.unique()):
                if " " + entity + " " in " " + text[paragraph][sentence] +  " ":
                    loc_start = text[paragraph][sentence].find(entity)
                    loc_end = loc_start + len(entity)
                    
                    to_append =   {"entity":entity, 
                                   "paragraph":paragraph, 
                                   "sentence":sentence,
                                   "loc_start":loc_start,
                                   "loc_end":loc_end}
                    
                    ENTITIES = ENTITIES.append(to_append, ignore_index = True)
    ENTITIES = ENTITIES.drop_duplicates()
    ENTITIES = ENTITIES.sort_values(by=["paragraph", "sentence", "loc_start"])
    ENTITIES = ENTITIES.reset_index(drop=True)
    return ENTITIES

In [13]:
ENTITIES = getEntityPositions(text, entities)

In [14]:
answer

,entity_1,entity_2,attitude,time
0,Россия,Австралия,neg,current
1,Австралия,Россия,neg,current
2,Обама,ИГ,neg,current
3,Запад,Россия,neg,current
4,Россия,Запад,neg,current
5,Россия,Сирия,neg,current
6,Франция,Россия,pos,current
7,author,Россия,neg,current
8,Обама,Асад,neg,current
9,author,Обама,neg,current


In [15]:
for entity in entities.unique():
    if entity not in ENTITIES.entity.unique():
        print entity

Москва
Шенген
Запад-Россия
Сирии-России
Лондон
Брюссель
прим. авт
Россию
Варшаве
Вашингтон
Киев
Фигаро
Татьяна Кастуева-Жан
Париж


In [16]:
ENTITIES.head(20)

,entity,paragraph,sentence,loc_start,loc_end
0,Мыкола Сирук,1,1,4,16
1,Россия,2,1,159,165
2,Украины,2,2,40,47
3,ООН,2,2,50,53
4,Юрий Сергеев,2,2,54,66
5,Twitter,2,2,77,84
6,Россия,2,2,154,160
7,РФ,2,2,184,186
8,Россия,3,1,39,45
9,Сирии,3,1,136,141


In [17]:
def distances(loc1, loc2):
    """
    returns distances from one entity to another
    """
    paragraph = ENTITIES.loc[loc2].paragraph - ENTITIES.loc[loc1].paragraph
    sentence = ENTITIES.loc[loc2].sentence - ENTITIES.loc[loc1].sentence
    return paragraph, sentence

Теперь нужно идти по табличке с выделенными сущностями и отбирать их по критериям для пар. Если именительный падеж - скорее всего, это актор, если в предложении всего одна сущность, возможно, это отношение автора к ней, если сущности в разных абзацах - они не связаны, и т. д. Сюда бы хорошо добавить синтакснет, чтобы понимать, чем в предложении является сущность.

In [18]:
def getMorphCase(word):
    p = morph.parse(u"аборт")[0]
    return p.tag.case

In [20]:
ENTITIES.head(20)

,entity,paragraph,sentence,loc_start,loc_end
0,Мыкола Сирук,1,1,4,16
1,Россия,2,1,159,165
2,Украины,2,2,40,47
3,ООН,2,2,50,53
4,Юрий Сергеев,2,2,54,66
5,Twitter,2,2,77,84
6,Россия,2,2,154,160
7,РФ,2,2,184,186
8,Россия,3,1,39,45
9,Сирии,3,1,136,141


In [25]:
def getSentimentCertainity(word):
    word = lemmatize(word)
    try:
        tone, certainty =  Rusentilex[["tone", "certainty"]][Rusentilex.word.isin([word])].values[0]
    except:
        try:
            tone, certainty =  Rusentilex[["tone", "certainty"]][Rusentilex.word.isin([word.encode("utf8")])].values[0]
        except:
            tone, certainty = "NaN", "NaN"
    return tone, certainty

In [179]:
def getPairs(ENTITIES=ENTITIES, text=text, n_entities=2):
    N_ENTITIES = n_entities # глобальный параметр, сколько сущностей будем перебирать для текущей
    PAIRS = pd.DataFrame(columns=["entity_1", "entity_2", "sentence"])
    ent_1_number = 0
    while ent_1_number < len(ENTITIES):

        entity_1 = ENTITIES.entity[ent_1_number]
        for ent_2_number in range(N_ENTITIES):
            try:            
                ent_2_number = ent_1_number+ent_2_number+1
                entity_2 = ENTITIES.entity[ent_2_number]
                par_dist, sent_dist = distances(ent_1_number, ent_2_number)
                if lemmatize(entity_1) != lemmatize(entity_2): # не равны друг другу
                    if par_dist == 0: # в одном абзаце
                        if sent_dist == 0: # в одном предложении
                            PAIRS = PAIRS.append({"entity_1":entity_1,
                                                  "entity_2":entity_2,
                                                  "sentence":text[ENTITIES.loc[ent_1_number].paragraph][ENTITIES.loc[ent_1_number].sentence]},
                                                  ignore_index=True)
                    else:
                        #ent_1_number+=1
                        PAIRS = PAIRS.append({"entity_1":"Author",
                                                  "entity_2":entity_1,
                                                  "sentence":text[ENTITIES.loc[ent_1_number].paragraph][ENTITIES.loc[ent_1_number].sentence]},
                                                  ignore_index=True)
#                 else:
#                     ent_2_number+=1


            except:
                continue
        ent_1_number += 1
        
    return PAIRS.drop_duplicates()

In [180]:
# cur_sentence = text[ENTITIES.loc[ent_1_number].paragraph][ENTITIES.loc[ent_1_number].sentence]
# cur_sentence = tknzr.tokenize(cur_sentence)
# for word in cur_sentence:
#     if (len(word)<3) or (word.isdigit()):
#         cur_sentence.remove(word)
# sentiment = []

# for word in cur_sentence:
#     sentiment.append(getSentimentCertainity(word)[0])

# if len([x for x in sentiment if x!="NaN"]) != 0:

In [181]:
print(loadText(10))

{Author, Unknown} Идеальный сценарий для Кремля

{Author, Unknown} By Мыкола Сирук, inosmi.ruПосмотреть оригиналноябрь 17-го, 2015


{Author, Unknown} Уже в первые минуты после парижских терактов в минувшую пятницу в экспертной среде стали раздаваться предположения, что от этой трагедии больше всего выиграет Россия. 
{Author, Unknown} В частности и постоянный представитель Украины в ООН Юрий Сергеев написал в Twitter: «В парижской трагедии «ищите женщину» — cherchez la femme — имя ее — Россия, ожидаем приглашений от РФ присоединиться к борьбе с террором».


{Author, Unknown} И первым тревожным признаком того, что Россия уже диктует свои условия, стало то, что из переговоров по мирному урегулированию ситуации в Сирии, была исключена Австралия. 
{Author, Unknown} В частности как сообщила британская газета The Guardian со ссылкой на источники в правительстве, Канберру исключили из переговоров, которые проходят под эгидой Международной группы поддержки Сирии именно по настоянию России. 
{A

In [228]:
PAIRS = getPairs(n_entities=3)

In [229]:
PAIRS.shape

(195, 3)

In [230]:
answer.head()

,entity_1,entity_2,attitude,time,pair
0,Россия,Австралия,neg,current,"россия,австралия"
1,Австралия,Россия,neg,current,"австралия,россия"
2,Обама,ИГ,neg,current,"обама,иго"
3,Запад,Россия,neg,current,"запад,россия"
4,Россия,Запад,neg,current,"россия,запад"


In [231]:
PAIRS = PAIRS.groupby(["entity_1", "entity_2"]).sum().reset_index()

In [232]:
answer["pair"] = answer.entity_1.apply(lambda x: lemmatize(x).lower())+","+\
                 answer.entity_2.apply(lambda x: lemmatize(x).lower())
    
answer["pair"] = answer["pair"].apply(lambda x: x.decode("utf8"))

In [233]:
PAIRS["pair"] = PAIRS.entity_1.apply(lambda x: lemmatize(x).lower())+","+\
                PAIRS.entity_2.apply(lambda x: lemmatize(x).lower())
#PAIRS["pair"] = PAIRS["pair"].apply(lambda x: x.encode("utf8"))

In [234]:
PAIRS[PAIRS.entity_1==u"Россия"].iloc[0].pair

u'\u0440\u043e\u0441\u0441\u0438\u044f,\u0430\u0432\u0441\u0442\u0440\u0430\u043b\u0438\u044f'

In [235]:
answer[answer.entity_1=="Россия"].loc[0].pair

u'\u0440\u043e\u0441\u0441\u0438\u044f,\u0430\u0432\u0441\u0442\u0440\u0430\u043b\u0438\u044f'

In [238]:
answer[answer.entity_1=="Россия"]

,entity_1,entity_2,attitude,time,pair
0,Россия,Австралия,neg,current,"россия,австралия"
4,Россия,Запад,neg,current,"россия,запад"
5,Россия,Сирия,neg,current,"россия,сирия"
12,Россия,Украина,neg,current,"россия,украина"


In [240]:
PAIRS[PAIRS.entity_1==u"Россия"].head()

,entity_1,entity_2,sentence,pair
122,Россия,Австралия,И первым тревожным признаком того что Россия ...,"россия,австралия"
123,Россия,Донбасс,Однако эти атаки не отменяют того факта что Р...,"россия,донбасс"
124,Россия,ЕС,Россия сейчас не играет на привлечение ЕС на ...,"россия,ес"
125,Россия,Запад,И выигрывают от нее исламисты а также Россия ...,"россия,запад"
126,Россия,Крым,Однако эти атаки не отменяют того факта что Р...,"россия,крым"


In [236]:
PAIRS_JOINED = PAIRS.join(answer.attitude, on="pair")

In [237]:
PAIRS_JOINED[PAIRS_JOINED.entity_1==u"Россия"]

,entity_1,entity_2,sentence,pair,attitude
122,Россия,Австралия,И первым тревожным признаком того что Россия ...,"россия,австралия",NaN
123,Россия,Донбасс,Однако эти атаки не отменяют того факта что Р...,"россия,донбасс",NaN
124,Россия,ЕС,Россия сейчас не играет на привлечение ЕС на ...,"россия,ес",NaN
125,Россия,Запад,И выигрывают от нее исламисты а также Россия ...,"россия,запад",NaN
126,Россия,Крым,Однако эти атаки не отменяют того факта что Р...,"россия,крым",NaN
127,Россия,ННГ,Татьяна КастуеваЖан руководитель центра Росси...,"россия,ннг",NaN
128,Россия,Париже,— Очевидно что Россия является наибольшим бен...,"россия,париж",NaN
129,Россия,РФ,В частности и постоянный представитель Украин...,"россия,рф",NaN
130,Россия,Сирии,И первым тревожным признаком того что Россия ...,"россия,сирия",NaN
131,Россия,Французском институте международных отношений,Татьяна КастуеваЖан руководитель центра Росси...,"россия,французский институт международный отно...",NaN


In [156]:
word = lemmatize(cur_sentence)

In [157]:
getSentimentCertainity(cur_sentence[16])[0]

'NaN'

In [159]:
getSentimentCertainity("трагедии")[0]

'negative'

In [160]:
np.nansum(np.abs(sentiment))

0.0

In [161]:
print(text[ENTITIES.loc[ent_1_number].paragraph][ENTITIES.loc[ent_1_number].sentence])

 В частности и постоянный представитель Украины в ООН Юрий Сергеев написал в Twitter «В парижской трагедии «ищите женщину» — cherchez la femme — имя ее — Россия ожидаем приглашений от РФ присоединиться к борьбе с террором»



In [162]:
getSentimentCertainity(cur_sentence[5])

('NaN', 'NaN')

In [163]:
ENTITIES.head(10)

,entity,paragraph,sentence,loc_start,loc_end
0,Мыкола Сирук,1,1,4,16
1,Россия,2,1,159,165
2,Украины,2,2,40,47
3,ООН,2,2,50,53
4,Юрий Сергеев,2,2,54,66
5,Twitter,2,2,77,84
6,Россия,2,2,154,160
7,РФ,2,2,184,186
8,Россия,3,1,39,45
9,Сирии,3,1,136,141


In [164]:
PAIRS.drop_duplicates().shape

(114, 4)

In [165]:
PAIRS_lemmatized = PAIRS.copy()
PAIRS_lemmatized["entity_1"] = PAIRS_lemmatized["entity_1"].apply(lambda x: lemmatize(x))
PAIRS_lemmatized["entity_2"] = PAIRS_lemmatized["entity_2"].apply(lambda x: lemmatize(x))

In [166]:
PAIRS_lemmatized[PAIRS_lemmatized.duplicated()].drop_duplicates()

,entity_1,entity_2,sentence,pair


In [167]:
answer

,entity_1,entity_2,attitude,time,pair
0,Россия,Австралия,neg,current,"россия,австралия"
1,Австралия,Россия,neg,current,"австралия,россия"
2,Обама,ИГ,neg,current,"обама,иго"
3,Запад,Россия,neg,current,"запад,россия"
4,Россия,Запад,neg,current,"россия,запад"
5,Россия,Сирия,neg,current,"россия,сирия"
6,Франция,Россия,pos,current,"франция,россия"
7,author,Россия,neg,current,"author,россия"
8,Обама,Асад,neg,current,"обама,асад"
9,author,Обама,neg,current,"author,обама"
